In [ ]:
'''
input: .svg vectors

output: gcodes - relative
    cur y = y1: 
        cur x = x1 => go to x2 with power
        cur x != x1 => go to x1, go to x2 with power
    next y:
        go to x1 y1, do (cur y = y1)
'''

'''
Hyperparams: size_scale, power_scale

Gcode config:
    G0 
    G
    G
    M
    M
'''

In [ ]:
import xml.etree.ElementTree as ET

In [ ]:


def svg2gcode(size_scale = 1.0, power_scale = 1.0):
    svg_file = "images/test1svg"
    tree = ET.parse(svg_file)
    root = tree.getroot()

    max_laser_power = 255
    feed_rate = 500 #?

    gcode = ["G21","G90", f"F{feed_rate}"] # unit: mm, position: absolute, feedrate?
    cur_x = 0
    cur_y = 0
    cur_laser = 0

    for line in root.findall(".//{http://www.w3.org/2000/svg}line"):
        x1 = float(line.attrib['x1']) * size_scale
        y1 = float(line.attrib['y1']) * size_scale
        x2 = float(line.attrib['x2']) * size_scale
        y2 = float(line.attrib['y2']) * size_scale
        opacity = float(line.attrib.get('opacity', 0.0))
        laser_power = int(opacity * max_laser_power * power_scale)

        if (cur_y != y1):
            1# move to y0
        if (cur_x != x1):
            1#move to x0
        else:
            if (laser_power != cur_laser):
                gcode.append(f"M3 S{laser_power}")
                gcode.append(f"G1 X{x2} Y{y2}")
            else:
                gcode.append(f"G1 X{x2} Y{y2}")
    gcode.append()# turnoff
    gcode.append("G0 X0 Y0")

    return "\n".join(gcode)

In [ ]:
gcode_output = svg2gcode()
with open("output.gcode", "w") as file:
    file.write(gcode_output)